In [ ]:
## sage implementation of bsgs testbench

import time
from sage.all import *
from sage.rings import *
from sage.groups.generic import *
from Crypto.Util.number import *
import matplotlib.pyplot as plt
import random

def generate_ecdh(primes_size):
    p = getPrime(prime_size)

    a = random.randint(1, p - 1)
    b = random.randint(1, p - 1)
    E = EllipticCurve(GF(p), [a, b])
    G = E.gens()[0]

    while E.order() != G.order():
        a = random.randint(1, p - 1)
        b = random.randint(1, p - 1)
        E = EllipticCurve(GF(p), [a, b])
        G = E.gens()[0]
    
    exp = random.randint(1, p - 1)
    B = exp * G

    return (G, exp, B)

def dlog_attack(G, exp, B):
    start_time = time.time()
    dlog = bsgs(G, B, (0, G.order() - 1), operation='+')
    end_time = time.time()

    if int(dlog) != int(exp):
        raise Exception(f"dlog failed for G = {generator}, exp = {exp}, B = {b}. computed dlog = {dlog}") 

    dlog_time = end_time - start_time 
    return (dlog, dlog_time)

prime_sizes = []
dlog_times = []

for prime_size in range(10, 75, 5): 
    print(f"Testing prime size: {prime_size}-bit")
    
    generator, exp, b = generate_ecdh(prime_size)
    print(f"Generated ECDH parameters: (generator, exp, b) = ({generator}, {exp}, {b})")
    
    dlog, dlog_time = dlog_attack(generator, exp, b)
    prime_sizes.append(prime_size)
    dlog_times.append(dlog_time)

    print(f"computed DLOG: {dlog}")
    print(f"Time taken to compute the DLOG: {dlog_time:.6f} seconds\n")

plt.plot(prime_sizes, dlog_times, marker='o', linestyle='-', color='b', label='DLOG time')
plt.xlabel('Prime Size (bits)')
plt.ylabel('Time Taken to compute DLOG (seconds)')
plt.title("Shank's BSGS Algorithm: Time to compute DLOG vs Prime Size")
plt.grid(True)
plt.legend()
plt.show()
